#generate sequence id's, ways of doing it
** monotonically increase id's <br>
** row_number() [windows function]<br>
** crc32 <br>
** md5 <br>
** sha1 and sha2

In [0]:
df_g2k = spark.read.option("nullvalue","null").csv("/FileStore/global2ksample.csv",header=True).dropna(how='all').dropDuplicates(['priority_rank'])

In [0]:
df_g2k.display()

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K


In [0]:
#generate sequence id using monotonically
from pyspark.sql.functions import monotonically_increasing_id
df_g2k = df_g2k.withColumn('id',monotonically_increasing_id())
#inn case we want to start the id from a specific number
df_g2k = df_g2k.withColumn('id',monotonically_increasing_id() +90)
df_g2k.orderBy(['priority_rank']).display()

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K,90
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K,91
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K,92
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K,93
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K,94
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K,95
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K,96
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K,97
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K,98
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K,99


generate sequence using <B>row_number<B>

In [0]:

#lit() is used to add a constant or literal value, row_number for partition
#window function to be separately imported
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
df_g2k = df_g2k.withColumn("row_number",row_number().over(Window.partitionBy(lit('')).orderBy(lit(''))))
df_g2k.display()

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id,row_number
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K,90,1
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K,91,2
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K,92,3
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K,93,4
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K,94,5
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K,95,6
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K,96,7
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K,97,8
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K,98,9
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K,99,10


create random sequence id using '<B>crc32<B>'

In [0]:
from pyspark.sql.functions import crc32,col
df_g2k = df_g2k.withColumn('crc32_key',crc32(col('priority_rank')))
df_g2k.display()
#it can generate duplicates for 100k or more records crc32 not recommended for large number of records

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id,row_number,crc32_key
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K,90,1,2212294583
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K,91,2,2707236321
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K,92,3,450215437
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K,93,4,1842515611
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K,94,5,4088798008
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K,95,6,2226203566
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K,96,7,498629140
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K,97,8,1790921346
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K,98,9,4194326291
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K,99,10,2366072709


create sequence id with a hash key usng md5

In [0]:
from pyspark.sql.functions import md5
df_g2k = df_g2k.withColumn("md5_key",md5('priority_rank'))
df_g2k.display()
#not suggested for large number of records, or more than 50-100 million

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id,row_number,crc32_key,md5_key
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K,90,1,2212294583,c4ca4238a0b923820dcc509a6f75849b
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K,91,2,2707236321,d3d9446802a44259755d38e6d163e820
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K,92,3,450215437,c81e728d9d4c2f636f067f89cc14862c
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K,93,4,1842515611,eccbc87e4b5ce2fe28308fd9f2a7baf3
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K,94,5,4088798008,a87ff679a2f3e71d9181a67b7542122c
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K,95,6,2226203566,e4da3b7fbbce2345d7772b0674a318d5
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K,96,7,498629140,1679091c5a880faf6fb5e6087eb1b2dc
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K,97,8,1790921346,8f14e45fceea167a5a36dedd4bea2543
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K,98,9,4194326291,c9f0f895fb98ab9159f51fd0297e236d
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K,99,10,2366072709,45c48cce2e2d7fbdea1afc51c7c6ad26


create sequence id using ,<B>sha2<B>

In [0]:
from pyspark.sql.functions import sha2
df_g2k = df_g2k.withColumn("sha2_key",sha2("priority_rank",256))
df_g2k.display()
#for 50-100 million can go with 256 if more than that go with 512

priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id,row_number,crc32_key,md5_key,sha2_key
1,China,ICBC,653710657,ICBC,440000,Financial,null,null,Banks,null,null,G2K,90,1,2212294583,c4ca4238a0b923820dcc509a6f75849b,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b
10,United States of America,Amazon,884745530,Amazon,1298000,Wholesale and Retail,null,null,Catalog & Mail Order Houses,null,null,G2K,91,2,2707236321,d3d9446802a44259755d38e6d163e820,4a44dc15364204a80fe80e9039455cc1608281820fe2b24f1e5233ade6af1dd5
2,United States of America,JPMorgan Chase,047675947,JPMorgan Chase,250000,Financial,null,null,Banks,null,null,G2K,92,3,450215437,c81e728d9d4c2f636f067f89cc14862c,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35
3,United States of America,Berkshire Hathaway,001024314,Berkshire Hathaway,360000,Financial,null,null,Investments,null,null,G2K,93,4,1842515611,eccbc87e4b5ce2fe28308fd9f2a7baf3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce
4,China,China Construction Bank,654504299,China Construction Bank,347156,Financial,null,null,Banks,null,null,G2K,94,5,4088798008,a87ff679a2f3e71d9181a67b7542122c,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a
5,Saudi Arabia,Saudi Arabian Oil Company (Saudi Aramco),643371628,Saudi Arabian Oil Company (Saudi Aramco),66800,Manufacturing,null,null,Oil & Gas Refining & Marketing,null,null,G2K,95,6,2226203566,e4da3b7fbbce2345d7772b0674a318d5,ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d
6,United States of America,Apple,060704780,Apple,147000,Manufacturing,null,null,Consumer Goods,null,null,G2K,96,7,498629140,1679091c5a880faf6fb5e6087eb1b2dc,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683
7,United States of America,Bank of America,055169452,Bank of America,213000,Financial,null,null,Banks,null,null,G2K,97,8,1790921346,8f14e45fceea167a5a36dedd4bea2543,7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451
8,China,Ping An Insurance Group,654077502,Ping An Insurance Group,362035,Insurance,null,null,Life Insurance,null,null,G2K,98,9,4194326291,c9f0f895fb98ab9159f51fd0297e236d,2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3
9,China,Agricultural Bank of China,653713891,Agricultural Bank of China,459000,Financial,null,null,Banks,null,null,G2K,99,10,2366072709,45c48cce2e2d7fbdea1afc51c7c6ad26,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7


#get individual column wise null record count

In [0]:
#first get all columns
#for i in df_g2k.columns
#select columns
from pyspark.sql.functions import count,col,when
#this display all column count
#df_g2k.select([ count(col(i)) for i in df_g2k.columns]).display()
#now to get only null columns
df_g2k.select([count(when(col(i).isNull(),i)).alias(i) for i in df_g2k.columns]).display()


priority_rank,country,name,gu_duns,gu_name,gu_employee,sector,hq_duns,hq_duns_name,center,site_duns,site_duns_name,managed,id,row_number,crc32_key,md5_key,sha2_key
0,0,0,0,0,0,0,10,10,0,10,10,0,0,0,0,0,0


reference: https://youtu.be/yKueGqJAgwM